In [1]:
# 安装所需的Python包
#%%capture
# 在Colab中跳过重启消息
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

In [ ]:
!pip install unsloth vllm
!pip install --upgrade pillow
# 如果在本地运行此笔记本，还需要安装`diffusers` 本次代码展示的是使用本地进行运行代码
!pip install diffusers
# 临时安装特定的TRL夜间版本
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b


  Cloning https://github.com/huggingface/trl.git (to revision e95f9fb74a3c3647b86f251b7e230ec51c64b72b) to /tmp/pip-req-build-gxkx_j0u
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-gxkx_j0u
  Running command git rev-parse -q --verify 'sha^e95f9fb74a3c3647b86f251b7e230ec51c64b72b'
  Running command git fetch -q https://github.com/huggingface/trl.git e95f9fb74a3c3647b86f251b7e230ec51c64b72b
  Running command git checkout -q e95f9fb74a3c3647b86f251b7e230ec51c64b72b
  Resolved https://github.com/huggingface/trl.git to commit e95f9fb74a3c3647b86f251b7e230ec51c64b72b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# 导入Unsloth库并进行初始化
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-15 00:46:48 __init__.py:190] Automatically detected platform cuda.


In [4]:
# 加载Llama 3.1 8B Instruct模型，并设置参数
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # 可以增加以获得更长的推理轨迹
lora_rank = 32 # 较大的rank意味着更智能，但速度较慢

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # LoRA 16bit时为False
    fast_inference = True, # 启用vLLM快速推理
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # 如果内存不足则减少
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # 选择任意大于0的数字！建议8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # 如果内存不足则移除QKVO
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # 启用长上下文微调
    random_state = 3407,
)

==((====))==  Unsloth 2025.2.5: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-bnb-4bit with actual GPU utilization = 59.57%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.57 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 2.5 GB. Also swap space = 6 GB.
WARNING 02-15 00:47:07 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-15 00:47:13 config.py:542] This model supports multiple tasks: {'reward', 'classify', 'score', 'generate', 'embed'}. Defaulting 

[W215 00:47:16.701052013 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 02-15 00:47:18 weight_utils.py:252] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-15 00:47:23 model_runner.py:1115] Loading model weights took 5.3541 GB
INFO 02-15 00:47:23 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-15 00:47:26 worker.py:267] Memory profiling takes 3.11 seconds
INFO 02-15 00:47:26 worker.py:267] the current vLLM instance can use total_gpu_memory (14.57GiB) x gpu_memory_utilization (0.60) = 8.68GiB
INFO 02-15 00:47:26 worker.py:267] model weights take 5.35GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 0.74GiB; the rest of the memory reserved for KV Cache is 2.54GiB.
INFO 02-15 00:47:26 executor_base.py:110] # CUDA blocks: 1298, # CPU blocks: 3072
INFO 02-15 00:47:26 executor_base.py:115] Maximum concurrency for 512 tokens per request: 40.56x
INFO 02-15 00:47:31 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occu

Capturing CUDA graph shapes: 100%|██████████| 23/23 [00:23<00:00,  1.04s/it]

INFO 02-15 00:47:55 model_runner.py:1562] Graph capturing finished in 24 secs, took 0.58 GiB
INFO 02-15 00:47:55 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 31.64 seconds



Unsloth 2025.2.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:


# 数据准备
import re
from datasets import load_dataset, Dataset

# 加载和准备数据集
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    """从XML格式的文本中提取答案"""
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    """从带有####标记的文本中提取答案"""
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# 获取gsm8k问题数据集
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # 加载数据集
    data = data.map(lambda x: { # 映射数据集以创建提示和答案
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    })
    return data

dataset = get_gsm8k_questions()

In [8]:

# 奖励函数
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """根据回答的正确性给予奖励"""
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    """检查回答是否为整数"""
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """检查回答是否符合严格的格式"""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """检查回答是否符合宽松的格式"""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    """计算XML格式的元素数量"""
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    """根据XML格式的元素数量给予奖励"""
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

In [9]:

# 训练模型

# 设置GRPO训练器和所有配置
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # 使用vLLM进行快速推理
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # 增加到4以获得更平滑的训练
    num_generations = 6, # 如果内存不足则减少
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # 设置为1以进行完整的训练
    max_steps = 20,
    save_steps = 20,
    max_grad_norm = 0.1,
    report_to = "none", # 可以使用Weights & Biases
    output_dir = "outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


In [10]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 20
 "-____-"     Number of trainable parameters = 83,886,080


-------------------- Question:
Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers? 
Answer:
100 
Response:
</reasoning>Assume Ahmed's grade in the remaining 9 assignments is x. Emily's total grade in the first 9 assignments is 92. We can represent this as 92 + the grade on the final assignment. Since the final assignment is worth the same as the other assignments, we can say the grade on the final assignment is 0.9x, since it is 90% of the total of the other assignments. Now, we can write the equation for the total grades as follows: 

92 + 0.9x = 1x + 90 

Subtracting 92 from both sides gives us 0.9x = x - 2. 
Subtracting 0.9x from both sides gives us 0 = 0.1x - 2. Subse

Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,0.000000,0.000000,196.500000,0.000000
2,0.000000,0.040667,0.099613,183.500000,0.000000
3,0.000000,-0.019833,0.035000,137.000000,0.000005
4,0.000000,0.083333,0.204124,186.666672,0.000011
5,0.000000,0.000000,0.000000,191.333344,0.000017
6,0.000000,-0.032667,0.050623,134.000000,0.000142
7,0.000000,0.779667,1.336288,138.333344,0.000620
8,0.000000,1.923667,0.899381,138.166672,0.000516
9,0.000000,0.000000,0.000000,163.833344,0.000085
10,0.000100,0.416667,1.020621,123.166672,0.002092


-------------------- Question:
The gauge on a water tank shows that the tank is 1/3 full of water. To fill the tank, 16 gallons of water are added. How many gallons of water does the tank hold when full? 
Answer:
24 
Response:
Let's break this problem down. 

If the tank is 1/3 full and 16 gallons are added to fill it, then the amount of water added is equal to the amount that was already in the tank (1/3 of the total capacity) plus the amount needed to fill it (2/3 of the total capacity). 

Let x be the capacity of the tank in gallons. 

The equation becomes x - (1/3)x + 16 = x.

To simplify the equation, we can combine the x terms: (2/3)x + 16 = x.

Next, subtract (2/3)x from both sides of the equation: (1/3)x = 16.

Now, multiply both sides of the equation by 3 to get rid of the fraction: x = 16 * 3

x = 48

The tank holds 48 gallons of water when full. 
Extracted:
Let's break this problem down. 

If the tank is 1/3 full and 16 gallons are added to fill it, then the amount of water 

TrainOutput(global_step=20, training_loss=2.3078780293062096e-05, metrics={'train_runtime': 929.1762, 'train_samples_per_second': 0.022, 'train_steps_per_second': 0.022, 'total_flos': 0.0, 'train_loss': 2.3078780293062096e-05})

In [11]:

# 推理
# 现在让我们尝试刚刚训练的模型！首先，尝试未经过GRPO训练的模型：

text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.62s/it, est. speed input: 1.65 toks/s, output: 20.07 toks/s]


"Calculating pi is a complex task that requires a combination of mathematical formulas and computational algorithms. Here are a few ways to calculate pi:\n\n**Method 1: Archimedes' Approximation**\n\nArchimedes, a Greek mathematician, used the Pythagorean theorem to approximate pi in the 3rd century BC. His method is based on the following formula:\n\npi (π) ≈ (a / c)\n\nwhere a and b are the lengths of the sides of a regular hexagon inscribed in a circle, and c is the diameter of the circle.\n\nUsing this method, Archimedes approximated pi as:\n\npi (π) ≈ 3.1418\n\n**Method 2: Leibniz Formula**\n\nGottfried Wilhelm Leibniz, a German mathematician, discovered a series of formulas for calculating pi in the 17th century. One of his formulas is:\n\npi (π) = 4 * (1 - 1/3 + 1/5 - 1/7 + 1/9 - ...)\n\nThis formula is an infinite series, and the more terms you add, the more accurate the approximation of pi will be.\n\n**Method 3: Calculus**\n\nUsing calculus, we can approximate pi using the fo

In [12]:
model.save_lora("grpo_saved_lora")

In [13]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.35s/it, est. speed input: 2.61 toks/s, output: 19.36 toks/s]


'The calculation of pi (π) is an ongoing effort in mathematics, and there is no single, definitive formula to compute it exactly. However, I can provide you with an approximation of pi using various methods.\n\nOne of the simplest methods to calculate pi is the Leibniz formula, which is a infinite series:\n\nπ/4 = 1 - 1/3 + 1/5 - 1/7 + 1/9 - ...\n\nThis series is an infinite sum, and the approximation gets better with more terms. However, this method is not the most efficient way to calculate pi.\n\nA more efficient method is the Bailey-Borwein-Plouffe (BBP) formula, which is a spigot algorithm for computing any base-b representation of pi:\n\nπ = Σ (1/(16^k)) * ((4/(8k+1)) - (2/(8k+4)) - (1/(8k+5)) - (1/(8k+6)))\n\nThis formula is more efficient than the Leibniz formula and can be used to calculate pi up to a large number of decimal places.\n\nAnother method to calculate pi is using the Monte Carlo method, which is based on random sampling:\n\nπ = (number of points inside the circle) 

In [16]:

# 保存为float16以用于VLLM
# 我们还支持直接保存为float16。选择merged_16bit用于float16或merged_4bit用于int4。我们还允许lora适配器作为后备。
# 使用push_to_hub_merged上传到您的Hugging Face账户！您可以访问https://huggingface.co/settings/tokens获取您的个人令牌。

import os  # 需要添加 os 模块
# 定义基础路径
base_path = os.path.join("/mnt/data1", "unsloth")
# 确保目录存在（可选添加）
os.makedirs(base_path, exist_ok=True)


# 合并到16bit
if True: 
        save_path = os.path.join(base_path, "model-16bit")
        model.save_pretrained_merged(save_path, tokenizer, save_method = "merged_16bit",)
if True: 
       model.push_to_hub_merged("amerlin/unslothllama", tokenizer, save_method = "merged_16bit", token = "")

# 合并到4bit
#if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
#if True: model.push_to_hub_merged("amerlin/unslothllama", tokenizer, save_method = "merged_4bit", token = "")

# 仅LoRA适配器
#if True: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
#if True: model.push_to_hub_merged("amerlin/unslothllama", tokenizer, save_method = "lora", token = "")


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 41.17 out of 78.04 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:29<00:00,  1.08it/s]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: You are pushing to hub, but you passed your HF username = amerlin.
We shall truncate amerlin/unslothllama to unslothllama


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 41.18 out of 78.04 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/amerlin/unslothllama


In [ ]:

# GGUF / llama.cpp 转换

# 现在我们支持本地保存到GGUF / llama.cpp！我们克隆llama.cpp并默认保存为q8_0。我们允许所有方法，如q4_k_m。使用save_pretrained_gguf进行本地保存，使用push_to_hub_gguf上传到HF。

# 一些支持的量化方法（完整列表在我们的Wiki页面上）：

# q8_0 - 快速转换。高资源使用，但通常可以接受。
# q4_k_m - 推荐。使用Q6_K用于一半的attention.wv和feed_forward.w2张量，其他使用Q4_K。
# q5_k_m - 推荐。使用Q6_K用于一半的attention.wv和feed_forward.w2张量，其他使用Q5_K。
# [NEW] 要微调并自动导出到Ollama，请尝试我们的Ollama笔记本

# 保存为8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# 记得访问https://huggingface.co/settings/tokens获取令牌！
# 并将hf更改为您的用户名！
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# 保存为16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# 保存为q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# 保存为多种GGUF选项 - 如果您想要多种选项，这样更快！
if False:
    model.push_to_hub_gguf(
        "hf/model", # 将hf更改为您的用户名！
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

In [ ]:
# 现在，使用model-unsloth.gguf文件或model-unsloth-Q4_K_M.gguf文件在llama.cpp或基于UI的系统中，如Jan或Open WebUI。您可以在此处安装Jan和Open WebUI

# 完成！如果您对Unsloth有任何疑问，我们有一个Discord频道！如果您发现任何错误或想要了解最新的LLM信息，或需要帮助，加入项目等，欢迎加入我们的Discord！

# 其他链接：

# Llama 3.2 会话笔记本。免费Colab
# 保存微调到Ollama。免费笔记本
# Llama 3.2 视觉微调 - 放射学用例。免费Colab
# 查看我们的文档中的DPO、ORPO、持续预训练、会话微调等笔记本！